# OSMO Deployment on Brev

This notebook walks through deploying a minimal OSMO on a Brev instance. 

If OSMO works for your use case here, it will scale to hundreds of GPUs in the cloud. You can use the exact same workflows; no code changes required.

⚠️ This deployment method is not recommended for production use as it lacks authentication and has limited features.⚠️

## Prerequisites (already installed)

- Docker
- KIND
- kubectl
- Helm
- Go
- nvkind
- Ansible (only for this experience)


## Step 1: Get Your Brev Instance's Public IP

Run this to get your instance's public IP address:


In [ ]:
curl -s ifconfig.me
echo ""

**Save this IP address!** You'll need it for the `/etc/hosts` configuration on your local machine.


## Step 2: Create KIND Cluster with GPU Support

Create the OSMO Kubernetes cluster using nvkind with the existing configuration:


In [ ]:
cd ~/OSMO/deployments/brev # navigate to example Brev directory

In [ ]:
nvkind cluster create --config-template=kind-osmo-cluster-config.yaml

## Step 3: Install GPU Operator

Install the GPU Operator to manage GPU Resources:

In [ ]:
helm fetch https://helm.ngc.nvidia.com/nvidia/charts/gpu-operator-v25.10.0.tgz
helm upgrade --install gpu-operator gpu-operator-v25.10.0.tgz \
  --namespace gpu-operator \
  --create-namespace \
  --set driver.enabled=false \
  --set toolkit.enabled=false \
  --set nfd.enabled=true \
  --wait

## Step 4: Install the KAI Scheduler

KAI scheduler provides co-scheduling, priority, and preemption for workflows:

In [ ]:
helm upgrade --install kai-scheduler \
  oci://ghcr.io/nvidia/kai-scheduler/kai-scheduler \
  --version v0.8.1 \
  --create-namespace -n kai-scheduler \
  --set global.nodeSelector.node_group=kai-scheduler \
  --set "scheduler.additionalArgs[0]=--default-staleness-grace-period=-1s" \
  --set "scheduler.additionalArgs[1]=--update-pod-eviction-condition=true" \
  --wait

## Step 5: Install OSMO

Deploy the complete OSMO platform with a single Helm command.
Installation takes about 5 minutes:

In [ ]:
helm fetch https://helm.ngc.nvidia.com/nvidia/osmo/charts/quick-start-1.0.0.tgz
helm upgrade --install osmo quick-start-1.0.0.tgz \
  --namespace osmo \
  --create-namespace \
  --wait

## Step 6: Configure Local Machine /etc/hosts

⚠️**Run this command on your local computer**⚠️ (not in the Brev instance):

```bash
echo "{ip} quick-start.osmo" | sudo tee -a /etc/hosts
```

Replace `{ip}` with the public IP from Step 1.

Example:
```bash
echo "203.0.113.45 quick-start.osmo" | sudo tee -a /etc/hosts
```

**Note:** If you stop/restart your Brev instance, it will get a new IP and you'll need to:
1. Remove the old entry from `/etc/hosts`
2. Add the new IP with this command again


## Step 7: Install OSMO CLI

Download and install the OSMO command-line interface:

In [ ]:
curl -fsSL https://raw.githubusercontent.com/NVIDIA/OSMO/refs/heads/main/install.sh | bash

## Step 8: Log in to OSMO

Authenticate with your local OSMO instance:

In [ ]:
osmo login http://quick-start.osmo --method=dev --username=testuser

## Step 9: Access the OSMO Web UI

Once the /etc/hosts is configured on your local machine, open a browser and navigate to:

```
http://quick-start.osmo
```

You should see the OSMO web interface.

## What to do next?
Try the [Hello World workflow](https://nvidia.github.io/OSMO/user_guide/main/tutorials/hello_world/index.html#tutorials-hello-world) or other [tutorials in the docs](https://nvidia.github.io/OSMO/user_guide/main/tutorials/overview.html).

## Cleanup
To remove the OSMO deployment:

In [ ]:
kind delete cluster --name osmo

⚠️ Remember to remove OSMO from /etc/hosts when finished with your Brev exploration ⚠️